# Assignment-21: Human activity detection [M]


In [2]:
# Importing Libraries

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [5]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [6]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [7]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [8]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [9]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

C:\Users\91889\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [12]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [13]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [14]:
# Initializing parameters
epochs = [30,35]
batch_size = 16
n_hidden = 32

In [15]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [16]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\91889\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

# 1) 32 LSTM with rmsprop optimizer

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
d=[0.25 ** x for x in range(1,3)]
for i in d:
    model.add(Dropout(i))    
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    model.summary()
    # Compiling the model
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    # Training the model
    for j in epochs:
        model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=j)
        
    




Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 71s 10ms/step - loss: 1.2928 - acc: 0.4580 - val_loss: 1.1458 - v

C:\Users\91889\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 66s 9ms/step - loss: 1.6325 - acc: 0.5362 - val_loss: 1.5337 - val_acc: 0.7282
Epoch 2/30
7352/7352 [==============================] - 66s 9ms/step - loss: 1.3362 - acc: 0.6757 - val_loss: 1.4824 - val_acc: 0.5555
Epoch 3/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.9996 - acc: 0.7368 - val_loss: 0.8736 - val_acc: 0.7937
Epoch 4/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.7847 - acc: 0.8259 - val_loss: 0.7836 - val_acc: 0.8626
Epoch 5/30
7352/7352 [==============================] - 62s 8ms/step - loss: 0.6316 - acc: 0.8734 - val_loss: 0.5730 - val_acc: 0.8948
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.5697 - acc: 0.8658 - val_loss: 0.7562 - val_acc: 0.8195
Epoch 7/

In [18]:
#best epoch - 30
#best dropout - 0.5

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))    
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [20]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2991 - acc: 0.4637 - val_loss: 1.3009 - val_acc: 0.3970
Epoch 2/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.0011 - acc: 0.5745 - val_loss: 0.9122 - val_acc: 0.5711
Epoch 3/30
7352/7352 [==============================] - 33s 4ms/step - loss: 0.9219 - acc: 0.5812 - val_loss: 0.8607 - val_acc: 0.6233
Epoch 4/30
7352/7352 [==============================] - 34s 5ms/step - loss: 0.7881 - acc: 0.6326 - val_loss: 0.7765 - val_acc: 0.6183
Epoch 5/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.7401 - acc: 0.6549 - val_loss: 0.7985 - val_acc: 0.6057
Epoch 6/30
7352/7352 [==============================] - 34s 5ms/step - loss: 0.7302 - acc: 0.6590 - val_loss: 0.8806 - val_acc: 0.5409
Epoch 7/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.6922 - acc: 0.6869 - val_loss: 0.7721 - val_acc: 0.6522
Epoch 8

In [21]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 515        0        22        0                   0   
SITTING                  1      374       114        1                   1   
STANDING                 0       76       454        2                   0   
WALKING                  0        0         0      466                  28   
WALKING_DOWNSTAIRS       0        0         0        1                 415   
WALKING_UPSTAIRS         0        0         0       50                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            2  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 403  


In [22]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 583us/step


In [23]:
score

[0.6073751995968245, 0.8914149983033594]

In [24]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(LSTM(32)) 
# Adding a dropout layer
model.add(Dropout(0.5))    
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 81s 11ms/step - loss: 1.1426 - acc: 0.4976 - val_loss: 1.3196 - val_acc: 0.3957
Epoch 2/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.7830 - acc: 0.6495 - val_loss: 0.7205 - val_acc: 0.6902
Epoch 3/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.6871 - acc: 0.7265 - val_loss: 0.6179 - val_acc: 0.7472
Epoch 4/30
7352/7352 [==============================] - 82s 11ms/step - loss: 0.5521 - acc: 0.7752 - val_loss: 0.5370 - val_acc: 0.7523
Epoch 5/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.4696 - acc: 0.7856 - val_loss: 0.6007 - val_acc: 0.7387
Epoch 6/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.4396 - acc: 0.7957 - val_loss: 0.4813 - val_acc: 0.7635
Epoch 7/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.3736 - acc: 0.8502 - val_loss: 0.5171 - val_acc: 0.8134


In [26]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 508        2        27        0                   0   
SITTING                  0      388       103        0                   0   
STANDING                 0      119       413        0                   0   
WALKING                  0        2         0      454                   6   
WALKING_DOWNSTAIRS       0        1         0        2                 413   
WALKING_UPSTAIRS         0       17         0        8                   7   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           34  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 439  
2947/2947 [==============================] - 3s 969us/step


[0.7223594442489476, 0.8873430607397353]

# 2) 64 LSTM (1 layer) with rmsprop optimizer

In [28]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 45s 6ms/step - loss: 1.2681 - acc: 0.4524 - val_loss: 1.2533 - val_acc: 0.4873
Epoch 2/30
7352/7352 [==============================] - 47s 6ms/step - loss: 1.0585 - acc: 0.5656 - val_loss: 0.9467 - val_acc: 0.5724
Epoch 3/30
7352/7352 [==============================] - 49s 7ms/step - loss: 0.8411 

In [29]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      371       118        0                   0   
STANDING                 0       95       433        1                   0   
WALKING                  0        0         0      431                  54   
WALKING_DOWNSTAIRS       0        0         0        0                 418   
WALKING_UPSTAIRS         0        0         0        2                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           3  
WALKING                           11  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 463  
2947/2947 [==============================] - 2s 788us/step


[0.39977529444468435, 0.8910756701730573]

# 3) 32 LSTM (2 layers) with rmsprop optimizer

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences=True,
               input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# second LSTM layer
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 30,598
Trainable params: 30,598
Non-trainable params: 0
_________________

In [18]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      375       115        0                   0   
STANDING                 0       68       464        0                   0   
WALKING                  0        0         0      467                  19   
WALKING_DOWNSTAIRS       0        0         0        0                 410   
WALKING_UPSTAIRS         0        0         0        2                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           10  
WALKING_DOWNSTAIRS                10  
WALKING_UPSTAIRS                 464  
2947/2947 [==============================] - 6s 2ms/step


[0.409253740882581, 0.9127926705123854]

 # 4) 64 LSTM (2 layer) with adam optimizer


In [19]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))  
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 157s 21ms/step - loss: 1.2012 - ac

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      376        91       17                   2   
STANDING                 0       91       433        8                   0   
WALKING                  0        0         0      463                  29   
WALKING_DOWNSTAIRS       0        0         0        5                 414   
WALKING_UPSTAIRS         0        0         0       24                  32   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            5  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 415  
2947/2947 [==============================] - 7s 2ms/step


[0.3044513650838959, 0.8951476077366813]

 # 5) 64 LSTM (2 layer) with adam optimizer, leaky relu activation


In [20]:
from keras.layers import LeakyReLU
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))  
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(LeakyReLU(alpha=0.001))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 6)                 0         
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________
Train 

 # 6) 62 LSTM (2 layer) with rms optimizer and Batch normalization


In [19]:
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences=True,
               input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# second LSTM layer
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=30)

C:\Users\91889\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)`
  from ipykernel import kernelapp as app


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
batch_normalization_1 (Batch (None, 6)                 24        
Total params: 30,622
Trainable params: 30,610
Non-trainable params: 12
_________________________________________________________________
Instr

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  STANDING
True                                
LAYING                 534         3
SITTING                  1       490
STANDING                 0       532
WALKING                  0       496
WALKING_DOWNSTAIRS       0       420
WALKING_UPSTAIRS         0       471
2947/2947 [==============================] - 4s 1ms/step


[8.404459636591433, 0.3617237869019342]

## Conclusion

In [21]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Model", "Layer" ,"AUC"]

x.add_row(["32 LSTM with rmsprop optimizer", 1 , 0.8914])
x.add_row(["64 LSTM with rmsprop optimizer", 1 , 0.8910])
x.add_row(["32 LSTM with rmsprop optimizer", 2 , 0.9127])
x.add_row(["64 LSTM with adam optimizer", 2 , 0.8951])
x.add_row(["64 LSTM with adam optimizer + relu activation", 2 , 0.4897])
x.add_row(["32 LSTM with rms optimizer + Batch Normalization", 2 , 0.3617])


print(x)

+--------------------------------------------------+-------+--------+
|                      Model                       | Layer |  AUC   |
+--------------------------------------------------+-------+--------+
|          32 LSTM with rmsprop optimizer          |   1   | 0.8914 |
|          64 LSTM with rmsprop optimizer          |   1   | 0.891  |
|          32 LSTM with rmsprop optimizer          |   2   | 0.9127 |
|           64 LSTM with adam optimizer            |   2   | 0.8951 |
|  64 LSTM with adam optimizer + relu activation   |   2   | 0.4897 |
| 32 LSTM with rms optimizer + Batch Normalization |   2   | 0.3617 |
+--------------------------------------------------+-------+--------+


- With a simple 1 layer architecture we got 89.1% accuracy.
- We can further improved the performace with Hyperparameter tuning.
- After hypertuning epoch and dropout_rate there was no major improvement in performance.
- After adding 1 more layer in the LSTM model, the accuracy improved slightly to 91.2% accuracy with a loss of 0.4.
- We tried to add batch normalization layer and Relu activation layer, however the performance deteriorated.

